In [11]:
# import pandas
import pandas as pd

d = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "latitude": 51.466652350000004,
    "longitude": -0.09686637642617651,
    "date": "2022-04-13",
    "current_time": "03:12:55.044",
    "sunrise": "06:09",
    "sunset": "19:53",
    "sun_status": "-",
    "solar_noon": "13:01",
    "day_length": "13:44",
    "sun_altitude": -23.19751117067553,
    "sun_distance": 1.4988500851835912E8,
    "sun_azimuth": 35.781559107335625,
    "moonrise": "15:43",
    "moonset": "05:28",
    "moon_status": "-",
    "moon_altitude": 20.615536932562232,
    "moon_distance": 387894.3437906608,
    "moon_azimuth": 266.5048405334666,
    "moon_parallactic_angle": 34.5669393631715
}


# Specify an index, for example, using the 'index' parameter
index_value = 1  # You can set an appropriate index value
df = pd.DataFrame(d, index=[index_value])

display(df)

,country,state,city,latitude,longitude,date,current_time,sunrise,sunset,sun_status,...,sun_altitude,sun_distance,sun_azimuth,moonrise,moonset,moon_status,moon_altitude,moon_distance,moon_azimuth,moon_parallactic_angle
1,United Kingdom,England,London,51.466652,-0.096866,2022-04-13,03:12:55.044,06:09,19:53,-,...,-23.197511,1.498850e+08,35.781559,15:43,05:28,-,20.615537,387894.343791,266.504841,34.566939


In [23]:
#pd.json_normalize
#As mentioned, this function is specifically designed to flatten nested JSON structures into a DataFrame.


import pandas as pd
from pandas import json_normalize

# Example nested JSON data
nested_json = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "population": 8908081,
    "area": {"total": 1572, "land": 1214, "water": 358},
    "moon": {"parallactic_angle": 34.5669393631715, "distance_km": 384400}
}

# Flatten the nested JSON using json_normalize
df = json_normalize(nested_json)

# Display the DataFrame
print(df)


          country    state    city  population  area.total  area.land  \
0  United Kingdom  England  London     8908081        1572       1214   

   area.water  moon.parallactic_angle  moon.distance_km  
0         358               34.566939            384400  


In [24]:
#explode function

#The explode function in pandas is used to transform lists-like columns in a DataFrame into separate
#rows, duplicating the rest of the data accordingly.If your JSON data has nested arrays that you want to convert to a tabular format, 
#you can use explode along with json_normalize. Here's an example:


import pandas as pd
from pandas import json_normalize

# Example JSON data with nested arrays
nested_json = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "population": 8908081,
    "languages": [{"name": "English", "speakers": 63000000},
                  {"name": "Welsh", "speakers": 562000}],
    "currencies": [{"name": "Pound Sterling", "code": "GBP"},
                   {"name": "Euro", "code": "EUR"}]
}

# Convert nested arrays to separate rows using json_normalize and explode
df = json_normalize(nested_json)
df_exploded = df.explode('languages').explode('currencies').reset_index(drop=True)

# Display the DataFrame
print(df_exploded)


          country    state    city  population  \
0  United Kingdom  England  London     8908081   
1  United Kingdom  England  London     8908081   
2  United Kingdom  England  London     8908081   
3  United Kingdom  England  London     8908081   

                                   languages  \
0  {'name': 'English', 'speakers': 63000000}   
1  {'name': 'English', 'speakers': 63000000}   
2      {'name': 'Welsh', 'speakers': 562000}   
3      {'name': 'Welsh', 'speakers': 562000}   

                                  currencies  
0  {'name': 'Pound Sterling', 'code': 'GBP'}  
1            {'name': 'Euro', 'code': 'EUR'}  
2  {'name': 'Pound Sterling', 'code': 'GBP'}  
3            {'name': 'Euro', 'code': 'EUR'}  


In [25]:
#Custom Function with apply: You can use the apply function to create custom functions that transform specific columns.
#This is useful when you have complex nesting.

def flatten_languages(row):
    return pd.Series(row['languages'][0])  # Assuming languages is a list with dictionaries

df = df.apply(flatten_languages, axis=1)

In [26]:
df

,name,speakers
0,English,63000000


In [28]:
#pd.DataFrame.from_records: If your nested data is a list of dictionaries, you can use from_records to create a DataFrame.

data = [{"name": "John", "age": 30}, {"name": "Alice", "age": 25}]
df = pd.DataFrame.from_records(data)
df

,name,age
0,John,30
1,Alice,25


In [30]:
#Manual Unpacking with apply:  If your data is deeply nested, you might need to manually unpack it using apply.

import pandas as pd

def unpack(row):
    # Extract the first language and currency from the lists
    primary_language = row['languages'][0] if row['languages'] else {}
    primary_currency = row['currencies'][0] if row['currencies'] else {}

    return pd.Series({
        'country': row['country'],
        'state': row['state'],
        'city': row['city'],
        'population': row['population'],
        'primary_language': primary_language.get('name', ''),
        'language_speakers': primary_language.get('speakers', ''),
        'primary_currency': primary_currency.get('name', ''),
        'currency_code': primary_currency.get('code', '')
    })

# Example nested JSON data
nested_json = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "population": 8908081,
    "languages": [{"name": "English", "speakers": 63000000},
                  {"name": "Welsh", "speakers": 562000}],
    "currencies": [{"name": "Pound Sterling", "code": "GBP"},
                   {"name": "Euro", "code": "EUR"}]
}

# Create a DataFrame from the nested JSON data
df = pd.DataFrame([nested_json])

# Apply the unpack function to each row
df_unpacked = df.apply(unpack, axis=1)

# Display the resulting DataFrame
print(df_unpacked)



          country    state    city  population primary_language  \
0  United Kingdom  England  London     8908081          English   

   language_speakers primary_currency currency_code  
0           63000000   Pound Sterling           GBP  


In [33]:
import pandas as pd

nested_json = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "population": 8908081,
    "languages": [{"name": "English", "speakers": 63000000},
                  {"name": "Welsh", "speakers": 562000}],
    "currencies": [{"name": "Pound Sterling", "code": "GBP"},
                   {"name": "Euro", "code": "EUR"}]
}

# Create a DataFrame from the nested JSON data
df = pd.DataFrame([{
    'country': nested_json['country'],
    'state': nested_json['state'],
    'city': nested_json['city'],
    'population': nested_json['population']
}])

# Display the resulting DataFrame
print(df)


          country    state    city  population
0  United Kingdom  England  London     8908081


In [42]:
import json

# Load JSON data

nested_json = {
    "country": "United Kingdom",
    "state": "England",
    "city": "London",
    "population": 8908081,
    "languages": [{"name": "English", "speakers": 63000000},
                  {"name": "Welsh", "speakers": 562000}],
    "currencies": [{"name": "Pound Sterling", "code": "GBP"},
                   {"name": "Euro", "code": "EUR"}]
}

data = pd.DataFrame(nested_json)
data



,country,state,city,population,languages,currencies
0,United Kingdom,England,London,8908081,"{'name': 'English', 'speakers': 63000000}","{'name': 'Pound Sterling', 'code': 'GBP'}"
1,United Kingdom,England,London,8908081,"{'name': 'Welsh', 'speakers': 562000}","{'name': 'Euro', 'code': 'EUR'}"


In [43]:
import pandas as pd
import json

json_data = '{"name": "John", "age": 25, "address": {"city": "New York", "state": "NY"}}'
data = json.loads(json_data)
df = pd.json_normalize(data)
print(df)


   name  age address.city address.state
0  John   25     New York            NY


In [48]:
import pandas as pd

# JSON data
json_data = {
    "name": "John, Jane",
    "age": 30,
    "city": "New York"
}

# Convert JSON data to DataFrame
df = pd.DataFrame([json_data])

# Write DataFrame to CSV
df.to_csv("output.csv", index=False)

In [51]:
import requests
import json
import pandas as pd

response = requests.get("https://api.example.com/employees")
employee_data = json.loads(response.text)
df = pd.DataFrame(employee_data)
df.to_csv("employee_data.csv", index=False)


ConnectionError: HTTPSConnectionPool(host='actual-api-url.com', port=443): Max retries exceeded with url: /employees (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f9752408520>: Failed to resolve 'actual-api-url.com' ([Errno -2] Name or service not known)"))

In [58]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [59]:
import json
from prettytable import PrettyTable

value = ['''
       {
           "_id": {
               "$Col1": "XXXXXXX2443"
           },
           "col2": false,
           "col3": "359335050111111",
           "startedAt": {
               "$date": 1633309625000
           },
           "endedAt": {
               "$date": 1633310213000
           },
           "col4": "YYYYYYYYYYYYYYYYYY",
           "created_at": {
               "$date": 1633310846935
           },
           "updated_at": {
               "$date": 1633310846935
           },
           "__v": 0
       }''']

dictionary = json.loads(value[0])
headers = []
values = []
for key in dictionary:
    head = key
    value = ""
    if type(dictionary[key]) == type({}):
        for key2 in dictionary[key]:
            head += "/" + key2
            value = dictionary[key][key2]
            headers.append(head)
            values.append(value)

    else:
        value = dictionary[key]
        headers.append(head)
        values.append(value)

print(headers)
print(values)
myTable = PrettyTable(headers)

myTable.add_row(values)
print(myTable)

['_id/$Col1', 'col2', 'col3', 'startedAt/$date', 'endedAt/$date', 'col4', 'created_at/$date', 'updated_at/$date', '__v']
['XXXXXXX2443', False, '359335050111111', 1633309625000, 1633310213000, 'YYYYYYYYYYYYYYYYYY', 1633310846935, 1633310846935, 0]
+-------------+-------+-----------------+-----------------+---------------+--------------------+------------------+------------------+-----+
|  _id/$Col1  |  col2 |       col3      | startedAt/$date | endedAt/$date |        col4        | created_at/$date | updated_at/$date | __v |
+-------------+-------+-----------------+-----------------+---------------+--------------------+------------------+------------------+-----+
| XXXXXXX2443 | False | 359335050111111 |  1633309625000  | 1633310213000 | YYYYYYYYYYYYYYYYYY |  1633310846935   |  1633310846935   |  0  |
+-------------+-------+-----------------+-----------------+---------------+--------------------+------------------+------------------+-----+


In [53]:
pip install json2table

Note: you may need to restart the kernel to use updated packages.


In [62]:
from json2table import convert

# Your JSON data
json_data = {
    "name": "John Doe",
    "age": 30,
    "city": "New York"
}

# Convert JSON to HTML table
table = convert(json_data)

# Print or display the HTML table
print(table)


<table><tr><th>name</th><td>John Doe</td></tr><tr><th>age</th><td>30</td></tr><tr><th>city</th><td>New York</td></tr></table>
